# Библиотека requests

Сначала разберемся, как получить информацию с внешнего сервиса. В стандартной библиотеке есть модуль urllib2, который может справиться с этой задачей. Однако большинство разработчиков используют стороннюю библиотеку requests, потому что в ней лучше реализованы большинство методов и код получается проще.

In [1]:
import requests

In [3]:
response = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
print(response)

<Response [200]>


Мы получили объект ответа, который содержит всю нужную нам информацию. По умолчанию на экран выводится HTTP-код ответа 200. Это означает, что запрос был корректным, и сервер отдал нам нужную информацию.

Код ответа в виде числа можно получить с помощью метода status_code:

In [5]:
print(response.status_code)

200


## Читаем результат

Мы сделали запрос и получили ответ. Давайте теперь посмотрим, как считывать текст. 

Адрес, по которому мы обращались, возвращает результат в json формате. Эти данные уже лежат в атрибуте text в полученном ответе response:

In [6]:
print(response.text)

{
    "Date": "2021-05-14T11:30:00+03:00",
    "PreviousDate": "2021-05-13T11:30:00+03:00",
    "PreviousURL": "\/\/www.cbr-xml-daily.ru\/archive\/2021\/05\/13\/daily_json.js",
    "Timestamp": "2021-05-13T23:00:00+03:00",
    "Valute": {
        "AUD": {
            "ID": "R01010",
            "NumCode": "036",
            "CharCode": "AUD",
            "Nominal": 1,
            "Name": "Австралийский доллар",
            "Value": 57.3364,
            "Previous": 57.7956
        },
        "AZN": {
            "ID": "R01020A",
            "NumCode": "944",
            "CharCode": "AZN",
            "Nominal": 1,
            "Name": "Азербайджанский манат",
            "Value": 43.7649,
            "Previous": 43.5786
        },
        "GBP": {
            "ID": "R01035",
            "NumCode": "826",
            "CharCode": "GBP",
            "Nominal": 1,
            "Name": "Фунт стерлингов Соединенного королевства",
            "Value": 104.4487,
            "Previous": 104.6703
 

Сейчас текст хранится просто в строковой переменной. Далее мы можем превратить эту строку в словарь. Сделать это можно с помощью JSON-парсера python, либо воспользовавшись методом json, который уже встроен в объект ответа response:

In [7]:
currencies = response.json()
print(currencies)

{'Date': '2021-05-14T11:30:00+03:00', 'PreviousDate': '2021-05-13T11:30:00+03:00', 'PreviousURL': '//www.cbr-xml-daily.ru/archive/2021/05/13/daily_json.js', 'Timestamp': '2021-05-13T23:00:00+03:00', 'Valute': {'AUD': {'ID': 'R01010', 'NumCode': '036', 'CharCode': 'AUD', 'Nominal': 1, 'Name': 'Австралийский доллар', 'Value': 57.3364, 'Previous': 57.7956}, 'AZN': {'ID': 'R01020A', 'NumCode': '944', 'CharCode': 'AZN', 'Nominal': 1, 'Name': 'Азербайджанский манат', 'Value': 43.7649, 'Previous': 43.5786}, 'GBP': {'ID': 'R01035', 'NumCode': '826', 'CharCode': 'GBP', 'Nominal': 1, 'Name': 'Фунт стерлингов Соединенного королевства', 'Value': 104.4487, 'Previous': 104.6703}, 'AMD': {'ID': 'R01060', 'NumCode': '051', 'CharCode': 'AMD', 'Nominal': 100, 'Name': 'Армянских драмов', 'Value': 14.2372, 'Previous': 14.1812}, 'BYN': {'ID': 'R01090B', 'NumCode': '933', 'CharCode': 'BYN', 'Nominal': 1, 'Name': 'Белорусский рубль', 'Value': 29.2639, 'Previous': 29.2556}, 'BGN': {'ID': 'R01100', 'NumCode': 

In [8]:
currencies['Valute']['UAH']

{'ID': 'R01720',
 'NumCode': '980',
 'CharCode': 'UAH',
 'Nominal': 10,
 'Name': 'Украинских гривен',
 'Value': 26.9106,
 'Previous': 26.7914}

In [9]:
print(currencies['Valute']['CZK']['Name'])

Чешских крон


## Оформляем функцию

В завершение давайте оформим наши вычисления в отдельную функцию, которой будет удобно пользоваться. На вход она должна принимать два параметра:

1. Название валюты currency. Например, 'EUR' или 'USD'.

2. Формат ответа format. При значении 'full' будем отдавать все, что знаем о валюте. Например, для currency = 'USD':

In [50]:
def exchange_rates(currency, format = 'full'):
    url = 'https://www.cbr-xml-daily.ru/daily_json.js'
    response = requests.get(url).json()['Valute']
    data = response[currency]
    if format == 'full':
        return data
    elif format == 'value':
        return data['Value']

In [51]:
exchange_rates('UAH', format = 'full')

'R01720'

## Задание 1

Напишите функцию currency_name, которая по ID валюты возвращает ее название на русском языке.

In [61]:
def currency_name(currency_code):
    url = 'https://www.cbr-xml-daily.ru/daily_json.js'
    response = requests.get(url).json()['Valute']
    for i in response:
        if response[i]['ID'] == currency_code:
            return  response[i]['Name']

In [64]:
currency_name('R01700J')

'Турецких лир'

# HTML-страницы

Довольно часто приходится добывать информацию не из удобно форматированного json-файла, а прямо с HTML-страниц. Получить содержимое страницы в большинстве случаев несложно, труднее извлечь из HTML-кода нужную информацию. В качестве примера мы рассмотрим страницу новости, из которой будем доставать полезную информацию: 1) заголовок страницы; 2) дату публикации; 3) текст публикации; 4) ссылки на странице.

In [65]:
url = 'https://nplus1.ru/news/2019/06/04/slothbot' 
response = requests.get(url)
print(response.status_code)

200


In [66]:
# Выведем полученные данные    
print(response.text)   

<!doctype html>
<html class="no-js bg-fixed _no-bg" style="background-image:url(https://nplus1.ru/images/2019/06/04/b32b62189fb87cce895e229e1d6d27b4.jpeg)" lang="">
<head>
    
    <meta charset="utf-8">
    <meta http-equiv="x-ua-compatible" content="ie=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1">
        <meta name="apple-mobile-web-app-capable" content="yes">
    <meta name="apple-mobile-web-app-status-bar-style" content="black">
                <meta name='wmail-verification' content='7991d7eb02d759f05b9050e111a7e3eb' />

    <link rel="canonical" href="https://nplus1.ru/news/2019/06/04/slothbot" />

        <title>Робота-ленивца научили лазать по паутине из тросов</title>

    	    <meta itemprop="datePublished" content="2019-06-04"/>
	
	    <meta name="mediator_author" content="Григорий Копиев"/> 
	
        <!-- amp page -->
    <link rel="amphtml" href="https://nplus1.ru/news/2019/06/04/slothbot/amp">
    

        <!-- for Google -->
        <

## Библиотека BeautifulSoup

Для поиска данных на странице воспользуемся библиотекой BeautifulSoup. Она позволяет по названию тэгов и их атрибутов получать содержащийся в них текст.

In [67]:
from bs4 import BeautifulSoup

In [83]:
# Получаем данные, как и ранее    
url = 'https://nplus1.ru/news/2019/06/04/slothbot'   
response = requests.get(url)

# Теперь создадим объект BeautifulSoup, указывая html парсер
page = BeautifulSoup(response.text, 'html.parser')

# Всё готово, чтобы получать данные из страницы    
# Для начала получим title, отображающийся на закладках браузера
print(page.title)  

<title>Робота-ленивца научили лазать по паутине из тросов</title>


In [84]:
# Мы получили тэг. Чтобы достать из него текст, вызовем атрибут text    
page.title.text    

'Робота-ленивца научили лазать по паутине из тросов'

Мы получили данные из title, но как нам забрать текст с самой страницы? Мы можем сделать это, запросив содержимое определённых тэгов. 

Пусть мы знаем, что заголовок статьи находится в тэге h1. Тогда мы можем получить его текст с помощью метода find

In [70]:
print(page.find('h1').text)

Робота-ленивца научили лазать по паутине из тросов


In [75]:
print(page.find('time').text)


18:27
04 Июнь 2019



## Задание 2 - Автоматический сбор заголовков

Напишите функцию wiki_header, которая по адресу страницы возвращает заголовок для статей на википедии

In [78]:
def wiki_header(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.find('h1').text
    return title

In [81]:
wiki_header('https://ru.wikipedia.org/wiki/Algorithm_(C%2B%2B)')

'algorithm (C++)'

## Больше данных

Больше данных
Мы рассмотрели базовый поиск конкретного элемента. Могут встречаться другие вариации:

когда нам нужно получить много похожих элементов; 
когда тэг атрибута неуникальный, тогда нам нужно использовать атрибуты (id, class), либо использовать вложенность.
Посмотрим на практике, что делать в случае, когда тэг элемента неуникальный. Пусть мы хотим получить сам текст статьи. Мы видим, что он находится в тэге div

Тэги div очень распространённые, их много на странице. Если мы просто используем find, то получим первый попавшийся, но это не то, что нам надо.

In [85]:
print(page.find('div').text)  

<img src = 'https://lms.skillfactory.ru/assets/courseware/v1/4f8c2e6cc99baf2cbd6c29828c1dd23a/asset-v1:Skillfactory+DST-PRO+15APR2020+type@asset+block/PY_9-9-1.png'>

Тут мы можем заметить, что у искомого текста есть свой класс GeneralMaterial-body. Воспользуемся этим:  в метод find можно передать аргумент class_

In [86]:
print(page.find('div', class_='body').text)







Allison Carter, Georgia Tech





Американские инженеры разработали робота-ленивца, способного лазать по тросам и перемещаться с одного троса на другой. Благодаря относительно малому потреблению энергии и использованию солнечных панелей подобного робота можно практически неограниченное время использовать для наблюдений на деревьях в лесу, рассказывают авторы статьи, представленной на конференции ICRA 2019.
Создание роботов, способных лазать по тросам — важная технологическая задача, наработки из которой потенциально применимы во многих областях. К примеру, в 2016 году в России создали робота для проверки высоковольтных линий электропередачи. Дрон спускает такого робота на грозозащитный трос, после чего тот самостоятельно перемещается вдоль основных проводов и осматривает их. Кроме того, потенциально таких роботов можно применять для исследований в густых лесах, в которых ветки соседних деревьев соприкасаются.
Инженеры из Технологического института Джорджии под руководством Магнус

## Сбор нескольких элементов

Собираем все ссылки на странице

Рассмотрим случай, когда нам нужно сразу много элементов. Пусть мы хотим получить название всех ссылок на странице в википедии про языки программирования.

Для ссылок существует тэг '<а></а>'.  Давайте попробуем использовать find  

In [87]:
url = 'https://en.wikipedia.org/wiki/List_of_programming_languages'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
page.find('a')

<a id="top"></a>

Что-то пошло не так, и мы получили только одну ссылку, хотя на странице их явно больше. Это происходит, потому что метод find возвращает только первый подходящий элемент. Если нам надо их больше, нужно воспользоваться методом find_all

In [88]:
links = page.find_all('a')

print(len(links))

927


In [89]:
# Посмотрим на некоторые из ссылок  
print([link.text for link in links[500:510]])  

['Orwell', 'Oxygene', 'Oz', 'edit', 'P', 'P4', 'P′′', 'ParaSail (programming language)', 'PARI/GP', 'Pascal']


Ещё одна полезная вещь:  последовательный поиск, т.е. мы можем найти сначала один элемент, а потом сделать внутри него второй поиск. Давайте выведем названия всех ссылок для языков программирования, которые начинаются на литеру "A".

In [91]:
# Получаем все элементы с тегом 'div' и классом 'div-col'  
all_blocks = page.find_all('div', class_='div-col')  

# Выбираем первый по счету блок  
first_block = all_blocks[0]  

# Берём оттуда ссылки (ограничимся первыми десятью)
links = first_block.find_all('a')
print([link.text for link in links[:10]])

['A.NET (A#/A sharp)', 'A-0 System', 'A+ (A (plus)', 'ABAP', 'ABC', 'ABC ALGOL', 'ACC', 'Accent (Rational Synergy)', 'Ace DASL (Distributed Application Specification Language)', 'Action!']


## HTML-таблицы

In [92]:
import pandas as pd

In [98]:
url = 'https://www.cbr.ru/key-indicators/'
pd.read_html(url)[1]

,0,1,2
0,рублей за грамм,13.05.2021,14.05.2021
1,Золото Au,"4 364,77","4 337,30"
2,Серебро Ag,6525,6412
3,Платина Pt,"2 939,84","2 895,04"
4,Палладий Pd,"6 996,11","6 837,17"


**Поиск таблицы с помощью BeautifulSoup**

Более надёжным решением является сначала получить только текст таблицы и передавать в pandas только её. Так же, как и в предыдущем разделе, используем requests и BeautifulSoup, чтобы получить данные со страницы.

А теперь — внимание! В случае работы с сайтом Центробанка нам нужно будет учесть ещё один нюанс. С недавнего времени администраторы стали блокировать обращения к сайту, поступающие не от «живых» пользователей, а от компьютерных программ (в том числе от той, которую мы собираемся написать прямо сейчас). Есть ли шанс в таких условиях всё-таки получить данные, не открывая страницу в браузере?

Безусловно, это возможно. Безвыходных положений вообще не бывает, особенно в программировании! Один из способов справиться с ситуацией — «притвориться» браузером :).

Каждый запрос, отправляемый с помощью команды get, имеет заголовок. Если заголовок не задан явно, то он создаётся автоматически, и туда добавляется информация в том числе о том, кто является субъектом (или источником) запроса — человек или программа. Чтобы «притвориться» браузером, нам просто нужно вручную дописать нужные сведения в этот заголовок.
https://habr.com/ru/company/ods/blog/346632/


In [99]:
from fake_useragent import UserAgent
UserAgent().chrome

'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36'

In [103]:
soup = BeautifulSoup(requests.get(url,headers = {'User-Agent': UserAgent().chrome}).text, 'html.parser')
soup


<!DOCTYPE html>

<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<meta content="telephone=no" name="format-detection">
<meta content="ru" name="zoom:lang">
<title>Ключевые показатели | Банк России</title>
<meta content="/common/images/share-1.jpg" property="og:image"/>
<link href="/common/libs/jquery-ui/jquery-ui.min.css?v=v3348762655" media="all" rel="stylesheet" type="text/css"/>
<!--[if IE 9]><link rel="stylesheet" type="text/css" href="/common/style/main-ie9.css?v=v2888097257" media="all"><![endif]-->
<!--[if gt IE 9]><!--><link href="/common/style/main.css?v=v3078198100" media="all" rel="stylesheet" type="text/css"/><!--<![endif]-->
<script src="/common/libs/jquery-3.4.1.min.js?v=v906141912" type="text/javascript"></script>
<script src="/common/libs/jquery-ui/jquery-ui.min.js?v=v906141912" type="

Далее нажимаем 'Исследовать элемент' в меню браузера на виджете с драгоценными металлами:

<img src = 'https://lms.skillfactory.ru/assets/courseware/v1/a2e35fa184cf7d6bbf2aac9a70f5fa71/asset-v1:Skillfactory+DST-PRO+15APR2020+type@asset+block/metals_2.png'>

Видим, что виджету соответствует следующая разметка:

<img src = 'https://lms.skillfactory.ru/assets/courseware/v1/1ed8cdbe78a85aa99b12651bf3044a89/asset-v1:Skillfactory+DST-PRO+15APR2020+type@asset+block/metals_3.png'>

Нам необходимо добраться до кода таблицы, который начинается с тэга <table>. Таблиц на странице много, поэтому указываем путь к таблице виджета драгоценных металлов применяя знания полученные ранее:

In [104]:
all_blocks = soup.findAll('div',class_='key-indicator_content offset-md-2')
data = all_blocks[1].find('table')

In [108]:
df = pd.read_html(str(data))[0]

In [109]:
df

,0,1,2
0,рублей за грамм,13.05.2021,14.05.2021
1,Золото Au,"4 364,77","4 337,30"
2,Серебро Ag,6525,6412
3,Платина Pt,"2 939,84","2 895,04"
4,Палладий Pd,"6 996,11","6 837,17"


## Задание 2 - Рейтинг банков

In [124]:
url = 'https://www.banki.ru/banks/ratings/'
soup = BeautifulSoup(requests.get(url,headers = {'User-Agent': UserAgent().chrome}).text, 'html.parser')

In [138]:
all_blocks = soup.findAll('div',class_='layout-column-full')

data = all_blocks[2].find('table')

In [139]:
df = pd.read_html(str(data))[0]

In [140]:
df

место                  название банка лицензия №, Регион  \
    место                  название банка лицензия №, Регион   
0       1            СберБанк лицензия № 1481, Москва и обл.   
1       2        ВТБ лицензия № 1000, Санкт-Петербург и обл.   
2       3          Газпромбанк лицензия № 354, Москва и обл.   
3       4  Национальный Клиринговый Центр лицензия № 3466...   
4       5          Альфа-Банк лицензия № 1326, Москва и обл.   
5       6      Россельхозбанк лицензия № 3349, Москва и обл.   
6       7  Московский Кредитный Банк лицензия № 1978, Мос...   
7       8       Банк Открытие лицензия № 2209, Москва и обл.   
8       9        Совкомбанк лицензия № 963, Костромская обл.   
9      10      Райффайзенбанк лицензия № 3292, Москва и обл.   
10     11             Росбанк лицензия № 2272, Москва и обл.   
11     12         ЮниКредит Банк лицензия № 1, Москва и обл.   
12     13               Траст лицензия № 3279, Москва и обл.   
13     14      Россия лицензия № 328, Санкт-Петербург и обл.   
14     15  Всероссийский Банк Развития Регионов лицензия ...   
15     16       Тинькофф Банк лицензия № 2673, Москва и обл.   
16     17  Санкт-Петербург лицензия № 436, Санкт-Петербур...   
17     18            Ситибанк лицензия № 2557, Москва и обл.   
18  19 +2         Банк ДОМ.РФ лицензия № 2312, Москва и обл.   
19     20         Новикомбанк лицензия № 2546, Москва и обл.   
20  21 +1            СМП Банк лицензия № 3368, Москва и обл.   
21  22 −3                 Ак Барс лицензия № 2590, Татарстан   
22     23          Банк Уралсиб лицензия № 30, Москва и обл.   
23     24             БМ-Банк лицензия № 2748, Москва и обл.   
24     25           Почта Банк лицензия № 650, Москва и обл.   
25     26  Московский Областной Банк лицензия № 1751, Мос...   
26  27 +1            Пересвет лицензия № 2110, Москва и обл.   
27  28 +1  Московский Индустриальный Банк лицензия № 912,...   
28  29 +1    Русский Стандарт лицензия № 2289, Москва и обл.   
29  30 −3  Национальный Расчетный Депозитарий лицензия № ...   
30  31 +1                  РНКБ лицензия № 1354, Симферополь   
31  32 −1  Уральский Банк Реконструкции и Развития лиценз...   
32     33        Абсолют Банк лицензия № 2306, Москва и обл.   
33     34     Хоум Кредит Банк лицензия № 316, Москва и обл.   
34     35          Банк Зенит лицензия № 3255, Москва и обл.   
35     36      Восточный Банк лицензия № 1460, Амурская обл.   
36     37            МТС Банк лицензия № 2268, Москва и обл.   
37     38    Транскапиталбанк лицензия № 2210, Москва и обл.   
38     39        Росэксимбанк лицензия № 2790, Москва и обл.   
39     40      Инвестторгбанк лицензия № 2763, Москва и обл.   
40     41            ИНГ Банк лицензия № 2495, Москва и обл.   
41     42           Локо-Банк лицензия № 2707, Москва и обл.   
42  43 +2        Сетелем Банк лицензия № 2168, Москва и обл.   
43  44 −1    Ренессанс Кредит лицензия № 3354, Москва и обл.   
44  45 +2            ОТП Банк лицензия № 2766, Москва и обл.   
45  46 +3        ВУЗ-Банк лицензия № 1557, Свердловская обл.   
46  47 +1            МСП Банк лицензия № 3340, Москва и обл.   
47  48 +2            Авангард лицензия № 2879, Москва и обл.   
48  49 −3         Возрождение лицензия № 1439, Москва и обл.   
49  50 +1  Азиатско-Тихоокеанский Банк лицензия № 1810, А...   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

# API